In [1]:
import os
import sys
import json
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

C:\Users\SUMSHEDHOKRISH\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import backend as K
from sklearn.metrics import precision_recall_fscore_support

In [4]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

FileNotFoundError: File b'train.csv' does not exist

In [4]:
data.shape


(95851, 8)

In [136]:
data.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [89]:
comments = data['comment_text']
comments_test = test['comment_text']
y_train=[]
for row in data.itertuples():
    y_train.append(list(row[3:9]))
y_test = []
for row in test.itertuples():
    y_test.append(list(row[3:9]))

In [48]:

input_list = list(comments)
input_list_test = list(comments_test)
type(input_list_test)
input_list_test = list(map(lambda x: str(x) , input_list_test))

In [50]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_list)
#tokenizer.fit_on_texts(input_list_test)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts

In [51]:
sequences = tokenizer.texts_to_sequences(input_list)
sequences_test = tokenizer.texts_to_sequences(input_list_test)

In [58]:
sequences.index(max(sequences,key = lambda x : len(x)))
len(max(sequences,key = lambda x : len(x)))

1403

In [59]:
len(vocab)

153188

In [61]:
pad_length = len(max(sequences, key = lambda x: len(x)))
pad_length

1403

In [62]:
sequences = pad_sequences(sequences, maxlen = pad_length)
sequences_test =  pad_sequences(sequences_test, maxlen = pad_length)

In [63]:
sequences

array([[    0,     0,     0, ...,    20,   350, 14246],
       [    0,     0,     0, ...,   170,    31,   121],
       [    0,     0,     0, ...,    15,    29,  1935],
       ..., 
       [    0,     0,     0, ...,    96,     1,  1066],
       [    0,     0,     0, ..., 63779,    32,    61],
       [    0,     0,     0, ...,    57,  9288,   194]])

In [94]:
y_train = np.array(y_train)
y_test = np.array(y_test)


In [101]:
len(sequences_test)

226998

In [17]:
BASE_DIR = ''

GLOVE_DIR = os.path.join(BASE_DIR)
print(GLOVE_DIR)

In [117]:
embeddings_index = {}

f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding='utf8')

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()

print('Found %s word vectors.' % len(embeddings_index))

# prepare embedding matrix
MAX_NB_WORDS = 10000
num_words = min(MAX_NB_WORDS, len(vocab) + 1)
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in vocab.items():

    if i >= MAX_NB_WORDS:

        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        # words not found in embedding index will be all-zeros.

        embedding_matrix[i] = embedding_vector
num_words

Found 400000 word vectors.


10000

In [122]:
model = Sequential()
model.add(Embedding(len(vocab)+1,100, input_length=pad_length,trainable=False))
model.add(LSTM(128,dropout=0.5,recurrent_dropout=0.2))
#model.add(LSTM(128,dropout=0.2,return_sequences=False))
model.add(Dense(6,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [123]:
history = model.fit(sequences[:30000], y_train[:30000], validation_split = 0.30, epochs = 3,batch_size=50)

Train on 21000 samples, validate on 9000 samples
Epoch 1/3
21000/21000 [==============================] - 14014s - loss: 0.1553 - acc: 0.9610 - val_loss: 0.1387 - val_acc: 0.9646
Epoch 2/3
14600/21000 [===================>..........] - ETA: 2976s - loss: 0.1414 - acc: 0.9634

KeyboardInterrupt: 

In [126]:
scores = model.evaluate(sequences[:10000], y_train[:10000], verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

10000/10000 [==============================] - 429s   
Accuracy: 96.29%


In [132]:
review = "Not that great. Don't buy"
review = [review]
#print(review)
to_check = tokenizer.texts_to_sequences(review)
#to_check = pad_sequences(to_check, maxlen=pad_length)
#to_check
review
to_check = np.array(to_check)
to_check = pad_sequences(to_check, maxlen = pad_length)

In [133]:
predict_class = model.predict(to_check)
print(predict_class)
label = np.argmax(predict_class) 
print("Class : ", label)


[[ 0.07836282  0.00830378  0.0406512   0.00320685  0.03209482  0.00809398]]
Class :  0
